In [2]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
import random

#Keras imports
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, AvgPool2D, Flatten, Dense
from keras.optimizers import Adam 
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

In [4]:
number_of_dataset_classes = 10
number_of_K_folds = 10

In [5]:
def get_dataset(dataset_name):
  if dataset_name == 'mnist':
    (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
  elif dataset_name == 'fasnion_minst':
    pass
  return (X_train, Y_train), (X_test, Y_test)

In [6]:
(X_train, Y_train), (X_test, Y_test) = get_dataset('mnist')

11493376/11490434 [==============================] - 0s 0us/step


In [7]:
def separate_dataset_into_K_folds(X_train, Y_train, number_of_K_folds):
  if number_of_K_folds == 10:
    folds = get_10_folds(X_train, Y_train)
  elif number_of_K_folds == 5:
    folds = get_5_folds(X_train, Y_train)
  return folds

In [8]:
def get_10_folds(X_train, Y_train, number_of_dataset_classes = number_of_dataset_classes):
  dataset_classes = get_dataset_classes(X_train, Y_train, number_of_dataset_classes)
  X_folds = [[],[],[],[],[],[],[],[],[],[]]
  Y_folds = [[],[],[],[],[],[],[],[],[],[]]
  
  #Pick each dataset class
  for jj, dataset_class in enumerate(dataset_classes):
    image_index = 0
    while image_index < len(dataset_class):
        for ii, fold in enumerate(X_folds):
          try:
            #print('image index is: ', image_index, 'dataset_class_index is: ', dataset_class_index)
            X_folds[ii].append(dataset_class[image_index])
            Y_folds[ii].append(jj)
            image_index += 1
          except Exception as e:
            continue
        
  #Convert X_folds and Y_folds to numpy arrays
  for ii, fold in enumerate(X_folds):
    X_folds[ii] = np.array(fold)
  for ii, fold in enumerate(Y_folds):
    Y_folds[ii] = np.array(fold)
  X_folds = np.array(X_folds)
  Y_folds = np.array(Y_folds)

  for ii, X_fold in enumerate(X_folds):
    Y_fold = Y_folds[ii]
    #c = np.array([X_fold, Y_fold])
    indices = np.arange(X_fold.shape[0])
    np.random.shuffle(indices)
    # c[0] = c[0][indices]
    # c[1] = c[1][indices]
    # X_fold, Y_fold = c[0], c[1]
    X_fold = X_fold[indices]
    Y_fold = Y_fold[indices]
    X_folds[ii] = X_fold
    Y_folds[ii] = Y_fold

  return X_folds, Y_folds

In [10]:
def get_5_folds(X_train, Y_train):
  pass

In [11]:
def get_dataset_classes(X_train, Y_train, number_of_dataset_classes):
  if number_of_dataset_classes == 10:
    dataset_classes = [[],[],[],[],[],[],[],[],[],[]]
  elif number_of_dataset_classes == 5:
    dataset_classes = [[],[],[],[],[]]
  for dataset_class_index in range(number_of_dataset_classes):
    for item in range(X_train.shape[0]):
      if Y_train[item] == dataset_class_index:
        dataset_classes[dataset_class_index].append(X_train[item])
  return np.array(dataset_classes)

In [12]:
X_folds, Y_folds = separate_dataset_into_K_folds(X_train, Y_train, 10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must speci

In [13]:
def iterate_over_folds(X_folds, Y_folds, number_of_K_folds):
  for ii, fold in enumerate(X_folds):
    X_stack = np.array([])
    Y_stack = np.array([])
    for i in range(number_of_K_folds):
      if i != ii:
        if X_stack.shape[0] == 0:
          X_stack = X_folds[i]
        else:
          X_stack = np.vstack((X_stack, X_folds[i]))
      if i != ii:
        if Y_stack.shape[0] == 0:
          Y_stack = Y_folds[i]
        else:
          Y_stack = np.vstack((Y_stack, Y_folds[i]))
    print(Y_stack)

In [14]:
iterate_over_folds(X_folds, Y_folds, number_of_K_folds)

ValueError: ignored

In [55]:
#https://medium.com/towards-artificial-intelligence/the-architecture-implementation-of-lenet-5-eef03a68d1f7
def create_lenet5_model_with_1_conv_layers():

  # Instanciate an empty model
  model = Sequential()

  # Adding a Convolution Layer C1
  # Input shape = N = (28 x 28)
  # No. of filters  = 6
  # Filter size = f = (5 x 5)
  # Padding = P = 0
  # Strides = S = 1
  # Size of each feature map in C1 is (N-f+2P)/S +1 = 28-5+1 = 24
  # No. of parameters between input layer and C1 = (5*5 + 1)*6 = 156
  model.add(Conv2D(filters=6, kernel_size=(5,5), padding='valid', input_shape=(28,28,1), activation='tanh'))

  # Adding an Average Pooling Layer S2
  # Input shape = N = (24 x 24)
  # No. of filters = 6
  # Filter size = f = (2 x 2)
  # Padding = P = 0
  # Strides = S = 2
  # Size of each feature map in S2 is (N-f+2P)/S +1 = (24-2+0)/2+1 = 11+1 = 12
  # No. of parameters between C1 and S2 = (1+1)*6 = 12
  model.add(AvgPool2D(pool_size=(2,2)))

  # Adding a Convolution Layer C3
  # Input shape = N = (12 x 12)
  # No. of filters  = 16
  # Filter size = f = (5 x 5)
  # Padding = P = 0
  # Strides = S = 1
  # Size of each feature map in C3 is (N-f+2P)/S +1 = 12-5+1 = 8
  # No. of parameters between S2 and C3 = (5*5*6*16 + 16) + 16 = 2416
  model.add(Conv2D(filters=16, kernel_size=(5,5), padding='valid', activation='tanh'))

  # Adding an Average Pooling Layer S4
  # Input shape = N = (8 x 8)
  # No. of filters = 16
  # Filter size = f = (2 x 2)
  # Padding = P = 0
  # Strides = S = 2
  # Size of each feature map in S4 is (N-f+2P)/S +1 = (8-2+0)/2+1 = 3+1 = 4
  # No. of parameters between C3 and S4 = (1+1)*16 = 32
  model.add(AvgPool2D(pool_size=(2,2)))

  # As compared to LeNet-5 architecture there was one more application of convolution but in our code  further application of 
  # convolution with (5 x 5) filter would result in a negative dimension which is not possible. So we aren't applying any more
  # convolution here.

  # Flattening the layer S4
  # There would be 16*(4*4) = 256 neurons
  model.add(Flatten())

  # Adding a Dense layer with `tanh` activation+# 
  # No. of inputs = 256
  # No. of outputs = 120
  # No. of parameters = 256*120 + 120 = 30,840
  model.add(Dense(120, activation='tanh'))

  # Adding a Dense layer with `tanh` activation
  # No. of inputs = 120
  # No. of outputs = 84
  # No. of parameters = 120*84 + 84 = 10,164
  model.add(Dense(84, activation='tanh'))

  # Adding a Dense layer with `softmax` activation
  # No. of inputs = 84
  # No. of outputs = 10
  # No. of parameters = 84*10 + 10 = 850
  model.add(Dense(10, activation='softmax'))

  #model.summary()

  return model

In [50]:
def create_lenet5_model_with_2_conv_layers():
  pass

In [51]:
def create_lenet5_model_with_3_conv_layers():
  pass

In [60]:
def compile_and_fit_model(model, train_x, train_y, val_x, val_y):
  #Reshape data
  train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
  val_x = val_x.reshape(val_x.shape[0], 28, 28, 1)

  #Normalize data
  train_x = train_x/255.0
  val_x = val_x/255.0

  #One-hot encode the labels
  train_y = to_categorical(train_y, num_classes=10)
  val_y = to_categorical(val_y, num_classes=10)

  model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
  model.fit(train_x, train_y, batch_size=128, epochs=20, verbose=1, validation_data=(val_x, val_y))
  return model

In [57]:
def evaluate_model(model, test_x, test_y):
  score = model.evaluate(test_x, test_y)
  print('Test Loss:', score[0])
  print('Test accuracy:', score[1])

In [58]:
model = create_lenet5_model_with_1_conv_layers()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 6)         156       
_________________________________________________________________
average_pooling2d (AveragePo (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
average_pooling2d_1 (Average (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               30840     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

In [61]:
model = compile_and_fit_model(model, X_train, Y_train, X_test, Y_test)

Epoch 1/20
469/469 [==============================] - 21s 44ms/step - loss: 0.6488 - accuracy: 0.8177 - val_loss: 0.1536 - val_accuracy: 0.9554
Epoch 2/20
469/469 [==============================] - 21s 44ms/step - loss: 0.1503 - accuracy: 0.9533 - val_loss: 0.1024 - val_accuracy: 0.9668
Epoch 3/20
469/469 [==============================] - 20s 44ms/step - loss: 0.0928 - accuracy: 0.9716 - val_loss: 0.0802 - val_accuracy: 0.9740
Epoch 4/20
469/469 [==============================] - 21s 44ms/step - loss: 0.0703 - accuracy: 0.9786 - val_loss: 0.0676 - val_accuracy: 0.9795
Epoch 5/20
469/469 [==============================] - 21s 44ms/step - loss: 0.0554 - accuracy: 0.9825 - val_loss: 0.0604 - val_accuracy: 0.9811
Epoch 6/20
469/469 [==============================] - 20s 44ms/step - loss: 0.0441 - accuracy: 0.9863 - val_loss: 0.0570 - val_accuracy: 0.9814
Epoch 7/20
469/469 [==============================] - 21s 44ms/step - loss: 0.0368 - accuracy: 0.9888 - val_loss: 0.0466 - val_accuracy:

In [31]:
def create_fold_iterables(X_folds, Y_folds):
  iterables = []
  for ii, val_fold in enumerate(X_folds):
    stack = 0
    for jj, train_fold in enumerate(X_folds):
      if ii != jj:
        if type(stack) is int:
          stack = train_fold
        else:
          stack= np.vstack((stack, train_fold))

    iterables.append([stack,val_fold,Y_folds[ii]])
  iterables = np.array(iterables)
  return iterables

In [32]:
i = create_fold_iterables(X_folds, Y_folds)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  del sys.path[0]


In [33]:
i

array([[array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 